# Common imports & library functions

In [44]:
import collections
from dataclasses import dataclass
import doctest
import functools
import itertools
import math

# Day 1: Report Repair

In [59]:
product = lambda ps: functools.reduce(lambda x, y: x * y, ps, 1)

def pick_numbers_that_sum_to(ns, pick_n, target):
    """
    Returns first `pick_n` numbers in `ns` to sum to `target`.
    >>> pick_numbers_that_sum_to([1, 9, 11, 3], 2, 12)
    (1, 11)
    >>> pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 2, 2020)
    (1721, 299)
    >>> pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 3, 2020)
    (979, 366, 675)
    """
    ns = [n for n in ns if n < target]
    return next(p for p in itertools.product(*[ns]*pick_n) if sum(p) == target)

In [41]:
doctest.run_docstring_examples(pick_numbers_that_sum_to, globs=None, verbose=True)

Finding tests in NoName
Trying:
    pick_numbers_that_sum_to([1, 9, 11, 3], 2, 12)
Expecting:
    (1, 11)
ok
Trying:
    pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 2, 2020)
Expecting:
    (1721, 299)
ok
Trying:
    pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 3, 2020)
Expecting:
    (979, 366, 675)
ok


In [46]:
# Final answers
with open('day1.txt') as f:
    ns = [int(l.strip()) for l in f]
    print('Part 1: ', product(pick_numbers_that_sum_to(ns, 2, 2020)))
    print('Part 2: ', product(pick_numbers_that_sum_to(ns, 3, 2020)))


Part 1:  876459
Part 2:  116168640


In [19]:
len([n for n in ns if n + 35 > 2020])

7

# Day 2: Password Philosophy

In [77]:
from dataclasses import dataclass
import re

line_re = re.compile(r'(\d+)-(\d+) (\w): (\w+)')

def parse_line(line):
    """
    >>> parse_line('1-3 a: abcde')
    ((1, 3, 'a'), 'abcde')
    """
    lo, hi, el, pw = line_re.match(line).groups()
    return ((int(lo), int(hi), el), pw)

def password_conforms_to_rule_original(rule, password):
    """
    >>> password_conforms_to_rule_original(*parse_line('1-3 a: abcde'))
    True
    >>> password_conforms_to_rule_original(*parse_line('1-3 b: cdefg'))
    False
    >>> password_conforms_to_rule_original(*parse_line('2-9 c: ccccccccc'))
    True
    """
    counts = collections.Counter(password)
    lo, hi, el = rule
    return lo <= counts.get(el, 0) <= hi

def password_conforms_to_rule_official(rule, password):
    """
    >>> password_conforms_to_rule_official(*parse_line('1-3 a: abcde'))
    True
    >>> password_conforms_to_rule_official(*parse_line('1-3 b: cdefg'))
    False
    >>> password_conforms_to_rule_official(*parse_line('2-9 c: ccccccccc'))
    False
    """
    lo, hi, el = rule
    return (password[lo-1] == el) ^ (password[hi-1] == el)

In [78]:
doctest.run_docstring_examples(parse_line, globs=None, verbose=True)
doctest.run_docstring_examples(password_conforms_to_rule_original, globs=None, verbose=True)
doctest.run_docstring_examples(password_conforms_to_rule_official, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_line('1-3 a: abcde')
Expecting:
    ((1, 3, 'a'), 'abcde')
ok
Finding tests in NoName
Trying:
    password_conforms_to_rule_original(*parse_line('1-3 a: abcde'))
Expecting:
    True
ok
Trying:
    password_conforms_to_rule_original(*parse_line('1-3 b: cdefg'))
Expecting:
    False
ok
Trying:
    password_conforms_to_rule_original(*parse_line('2-9 c: ccccccccc'))
Expecting:
    True
ok
Finding tests in NoName
Trying:
    password_conforms_to_rule_official(*parse_line('1-3 a: abcde'))
Expecting:
    True
ok
Trying:
    password_conforms_to_rule_official(*parse_line('1-3 b: cdefg'))
Expecting:
    False
ok
Trying:
    password_conforms_to_rule_official(*parse_line('2-9 c: ccccccccc'))
Expecting:
    False
ok


In [79]:
# Final answers
with open('day2.txt') as f:
    rules_and_pws = [parse_line(l.strip()) for l in f]
    original_pws = [l for l in rules_and_pws if password_conforms_to_rule_original(*l)]
    official_pws = [l for l in rules_and_pws if password_conforms_to_rule_official(*l)]
    print('Part 1: ', len(original_pws))
    print('Part 2: ', len(official_pws))

Part 1:  445
Part 2:  491


# Day 3: Toboggan Trajectory

In [56]:
from itertools import count

TREE = '#'

@dataclass
class Map:
    map: str
    w: int
    h: int

    def at(self, x, y):
        return self.map[(self.w + 1) * (y % self.h) + (x % self.w)]

def parse_map(map):
    clean_map = map.strip().replace('\n', '|')
    h = clean_map.count('|') + 1
    w = clean_map.index('|')
    return Map(map=clean_map, h=h, w=w)

def num_trees_hit(dx, dy, map):
    hits = 0
    for x, y in zip(count(0, dx), range(0, map.h, dy)):
        if map.at(x, y) == TREE:
            hits += 1
    return hits

In [57]:
test_map = parse_map("""
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
""")
assert num_trees_hit(dx=3, dy=1, map=test_map) == 7, 'test failed'

In [60]:
# Final answers
with open('day3.txt') as f:
    map = parse_map(f.read())
    print('Part 1: ', num_trees_hit(dx=3, dy=1, map=map))
    product_of_trees_hit = product(num_trees_hit(dx, dy, map)
                                   for dx, dy in ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2)))
    print('Part 2: ', product_of_trees_hit)

Part 1:  151
Part 2:  7540141059
